In [3]:
file_path

'/content/drive/MyDrive/.shortcut-targets-by-id/1SvKPuuNCR0sAS2Vdv1rmN7eApXBVP4Qp/'

In [1]:
  # @title Импорт библиотек и данных
  import os, sys, requests, warnings, pandas as pd, numpy as np
  from datetime import datetime as dt, timedelta
  from sqlalchemy import create_engine, text
  from bs4 import BeautifulSoup
  from tqdm import tqdm
  import warnings
  warnings.filterwarnings('ignore')
  np.set_printoptions(formatter={'float': lambda x: '{:,.2f}'.format(x).replace(',', ' ')}) # чтобы печатал красиво
  pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x).replace(',', ' ')) # чтобы печатал красиво
  from google.colab import drive

  drive.mount('/content/drive', force_remount=True)
  folder_id = '1SvKPuuNCR0sAS2Vdv1rmN7eApXBVP4Qp'
  file_path = f'/content/drive/MyDrive/.shortcut-targets-by-id/{folder_id}/'
  death_tab = pd.read_excel(file_path + 'Актуарная система ОПС и НПО.xlsx')
  year_month = pd.read_excel(file_path + 'Input/Год_месяц.xlsx')
  sys.path.append(file_path)
  import Create_excel as crex
  death_props = death_tab.iloc[:, :5]
  KBD_data = death_tab.iloc[:, 8:13]

Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/.shortcut-targets-by-id/1SvKPuuNCR0sAS2Vdv1rmN7eApXBVP4Qp/Актуарная система ОПС и НПО.xlsx'

In [ ]:
model_points = pd.read_excel(file_path + 'Модельные_точки.xlsx', sheet_name = 'НПО').query("Этап == 'Накопления' and Возраст < 55").drop(columns = 'Этап').reset_index(drop = True)

In [ ]:
for i, row in model_points.iloc[0:2, :].iterrows():
  Summary_all = NPO_model(row['Пол'], row['Возраст'], row['Тип_пенсии'], row['Остаток'], row['Пенсия'], i)[0].drop(columns = 'Сумма')
Summaries = []
t = dt.now()

for i, row in model_points.iterrows():
  if not i%10:
    print(f'Выполнено {i} модельных точек, время выполнения {(dt.now() - t).seconds} секунд')
  Summary = calc_all(row['Пол'], row['Возраст'], row['Тип_пенсии'], row['Остаток'], row['Пенсия'], i)
  Summaries.append(pd.DataFrame(Summary.rename(columns = {'Сумма':'Модельная точка ' + str(i)})['Модельная точка ' + str(i)]))
  Summaries.append(pd.DataFrame(Summary.rename(columns = {'Сумма помесячно':'Модельная точка ' + str(i) + ' помесячно'})['Модельная точка ' + str(i) + ' помесячно']))
Final = pd.concat([Summary_all] + Summaries, axis = 1).T
save_dir = file_path + "Итог НПО все точки.xlsx"
with pd.ExcelWriter(save_dir, engine = 'openpyxl') as writer:
  Final.to_excel(writer, sheet_name = 'Свод итогов', index = False)
crex.format_output(writer, save_dir)

Выполнено 0 модельных точек, время выполнения 0 секунд
Выполнено 10 модельных точек, время выполнения 80 секунд
Выполнено 20 модельных точек, время выполнения 130 секунд
Выполнено 30 модельных точек, время выполнения 180 секунд
Выполнено 40 модельных точек, время выполнения 224 секунд
Выполнено 50 модельных точек, время выполнения 305 секунд
Выполнено 60 модельных точек, время выполнения 351 секунд
Выполнено 70 модельных точек, время выполнения 398 секунд
Выполнено 80 модельных точек, время выполнения 439 секунд
Выполнено 90 модельных точек, время выполнения 519 секунд
Выполнено 100 модельных точек, время выполнения 567 секунд
Выполнено 110 модельных точек, время выполнения 614 секунд
Выполнено 120 модельных точек, время выполнения 657 секунд
Выполнено 130 модельных точек, время выполнения 739 секунд
Выполнено 140 модельных точек, время выполнения 785 секунд
Выполнено 150 модельных точек, время выполнения 829 секунд


In [ ]:
# @title Функция расчёта
def NPO_model(gender, age, Pension_type, Ostatok, Vznos, mt_num, lapce = 0.005, death = 0.0, system = 'new'):
  Admin = 2500.0
  Vznos = 1000.0
  # % Доходностей:
  brutto = 0.0
  inflation = 0.042
  # % Вознаграждений от СЧА
  VF_pct, UK_pct, SD_pct, ASV_pct = 0.005, 0.001, 0.003, 0.000125
  # % Вознаграждений от ИД
  VF_inv_pct, UK_inv_pct = 0.15, 0.03
  # Прочее
  Tarif = 11
  Pens_Term = 10
  # Расчётные параметры:
  DoR = 60 - age if gender == 'М' else 55 - age
  DoTO = DoR + Pens_Term if Pension_type in ['ДИ', 'СР'] else DoR if Pension_type == 'ЕВ' else 100

  # 1) Исходные данные таблицы смертности и данные ставок КБД
  calc_table = death_props.query("`Age (Y)` >= @age and `Age (Y)` < 111").rename(columns = {'Age (Y)' :'Age'})
  calc_table = calc_table[['Age', 'q_x (M)']] if age == 'M' else calc_table[['Age', 'q_x (F)']]
  calc_table = calc_table.rename(columns = {calc_table.columns[-1] : 'Death'})
  calc_table['Год'] = range(calc_table.shape[0])
  calc_table['noterm'] = (1-lapce)**calc_table['Год']

  calc_table = calc_table.merge(KBD_data[['Y','МСФО 17']].rename(columns = {'Y':'Год', 'МСФО 17':'brutto'}), how = 'left', on = 'Год')
  calc_table['live'] = 1.0
  for i in range(1, len(calc_table)):
      calc_table.loc[i, 'live'] = (1-calc_table.loc[i, 'Death'])*calc_table.loc[i-1, 'live']
  calc_table['netto'] = calc_table['brutto']*(1-VF_inv_pct) - ASV_pct - VF_pct if system == 'new' else \
    (calc_table['brutto']*(1-UK_inv_pct) - VF_pct - UK_pct - SD_pct - ASV_pct)*(1-VF_inv_pct)
  calc_table = calc_table[['Год', 'Age', 'noterm', 'Death', 'live', 'brutto', 'netto']]
  calc_table['Discont factor'] = 1.0
  for i in range(1, len(calc_table)):
    calc_table.loc[i, 'Discont factor'] = calc_table.loc[i - 1, 'Discont factor'] / (1 + calc_table.loc[i, 'brutto'])
  # 2) Расчёты нулевого периода
  calc_table.at[0, 'Остаток EOP'] = Ostatok

  # 3) Расчёты до выхода на пенсию
  for i in range(1, DoR + 1):
    calc_table.at[i, 'Остаток BOP'] = calc_table.at[i - 1, 'Остаток EOP']
    calc_table.at[i, 'ИД brutto sum'] = calc_table.at[i, 'Остаток BOP'] * calc_table.at[i, 'brutto']
    calc_table.at[i, 'ИД brutto prob'] = calc_table.at[i, 'ИД brutto sum'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'ИД netto prob'] = calc_table.at[i, 'Остаток BOP'] * calc_table.at[i, 'netto'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'ВФ пост prob'] = VF_pct * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'УК усп prob'] = UK_inv_pct * calc_table.at[i, 'ИД brutto prob']
    calc_table.at[i, 'УК упр prob'] = UK_pct * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'СД prob'] = SD_pct * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'АСВ prob'] = ASV_pct * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'ВФ перем prob'] = VF_inv_pct * (calc_table.at[i, 'ИД brutto prob'] if system == 'new' else calc_table.at[i, 'ИД brutto prob'] - \
      calc_table.loc[i, ['ВФ пост prob', 'УК усп prob', 'УК упр prob', 'СД prob', 'АСВ prob']].sum())

    calc_table.at[i, 'Взносы sum'] = Vznos
    calc_table.at[i, 'Взносы prob'] = calc_table.at[i, 'Взносы sum'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'Админ sum'] = Admin * (1 + inflation)**(calc_table.at[i-1, 'Год'])
    calc_table.at[i, 'Админ prob'] = calc_table.at[i, 'Админ sum'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']

    calc_table.at[i, 'Остаток MOP'] = calc_table.at[i, 'Остаток BOP'] * (1 + calc_table.at[i, 'netto']) + calc_table.at[i, 'Взносы sum']
    calc_table.at[i, 'Наследники sum'] = calc_table.at[i, 'Остаток MOP']
    calc_table.at[i, 'Наследники prob'] = calc_table.at[i, 'Наследники sum'] * calc_table.at[i, 'Death'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']*(1-lapce/2)
    calc_table.at[i, 'Выкупная sum'] = calc_table.at[i, 'Остаток MOP']
    calc_table.at[i, 'Выкупная prob'] = calc_table.at[i, 'Выкупная sum'] * lapce * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm'] * (1 - calc_table.at[i, 'Death']/2)
  # Прочее
    for cl in ['ЕВ sum', 'ЕВ prob', 'ПЖ sum', 'ПЖ prob', 'СР sum', 'СР prob', 'ДИ sum', 'ДИ prob']:
      calc_table.at[i, cl] = 0.0
    calc_table.at[i, 'Остаток EOP'] = max(0, calc_table.at[i, 'Остаток MOP'])
  # 4) Расчёты в момент выхода на пенсию
  calc_table.at[DoR, 'ЕВ sum'] = calc_table.at[DoR, 'Остаток MOP'] if Pension_type == 'ЕВ' else 0.0
  calc_table.at[DoR, 'ЕВ prob'] = calc_table.at[DoR, 'ЕВ sum'] * calc_table.at[DoR, 'live'] * calc_table.at[DoR, 'noterm']
  calc_table.at[DoR, 'ПЖ sum'] = calc_table.at[DoR, 'Остаток MOP'] / Tarif if Pension_type == 'ПЖ' else 0.0
  calc_table.at[DoR, 'ПЖ prob'] = calc_table.at[DoR, 'ПЖ sum'] * calc_table.at[DoR, 'live'] * calc_table.at[DoR, 'noterm']
  calc_table.at[DoR, 'СР sum'] = calc_table.at[DoR, 'Остаток MOP'] / Pens_Term if Pension_type == 'СР' else 0.0
  calc_table.at[DoR, 'СР prob'] = calc_table.at[DoR, 'СР sum'] * calc_table.at[DoR, 'live'] * calc_table.at[DoR, 'noterm']
  calc_table.at[DoR, 'ДИ sum'] = calc_table.at[DoR, 'Остаток MOP'] / Pens_Term if Pension_type == 'ДИ' else 0.0
  calc_table.at[DoR, 'ДИ prob'] = calc_table.at[DoR, 'ДИ sum'] * calc_table.at[DoR, 'live'] * calc_table.at[DoR, 'noterm']
  calc_table.at[DoR, 'Остаток EOP'] = max(0, calc_table.at[DoR, 'Остаток MOP'] - calc_table.loc[DoR, ['ЕВ sum', 'ПЖ sum', 'СР sum', 'ДИ sum']].sum())
  # calc_table.loc[16, :]
  i = 16
  Pens_Term - (calc_table.at[i, 'Год'] - DoR), DoR
  # 5) Расчёты после выхода на пенсию
  for i in range(DoR + 1, DoTO):
    calc_table.at[i, 'noterm'] = calc_table.at[i - 1, 'noterm']
    calc_table.at[i, 'Остаток BOP'] = calc_table.at[i - 1, 'Остаток EOP']
    calc_table.at[i, 'ИД brutto sum'] = calc_table.at[i, 'Остаток BOP'] * calc_table.at[i, 'brutto']
    calc_table.at[i, 'ИД brutto prob'] = calc_table.at[i, 'ИД brutto sum'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'ИД netto prob'] = calc_table.at[i, 'Остаток BOP'] * calc_table.at[i, 'netto'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'ВФ пост prob'] = VF_pct * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'УК усп prob'] = UK_inv_pct * calc_table.at[i, 'ИД brutto prob']
    calc_table.at[i, 'УК упр prob'] = UK_pct * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'СД prob'] = SD_pct * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'АСВ prob'] = ASV_pct * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'ВФ перем prob'] = VF_inv_pct * (calc_table.at[i, 'ИД brutto prob'] if system == 'new' else calc_table.at[i, 'ИД brutto prob'] - \
      calc_table.loc[i, ['ВФ пост prob', 'УК усп prob', 'УК упр prob', 'СД prob', 'АСВ prob']].sum())

    calc_table.at[i, 'Взносы sum'] = 0.0
    calc_table.at[i, 'Взносы prob'] = 0.0

    calc_table.at[i, 'Остаток MOP'] = calc_table.at[i, 'Остаток BOP'] * (1 + calc_table.at[i, 'netto']) + calc_table.at[i, 'Взносы sum']
    calc_table.at[i, 'Наследники sum'] = calc_table.at[i, 'Остаток MOP'] if Pension_type != 'ПЖ' else 0
    calc_table.at[i, 'Наследники prob'] = calc_table.at[i, 'Наследники sum'] * calc_table.at[i, 'Death'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'Выкупная sum'] = 0.0
    calc_table.at[i, 'Выкупная prob'] = 0.0

    calc_table.at[i, 'ЕВ sum'] = 0.0
    calc_table.at[i, 'ЕВ prob'] = 0.0
    calc_table.at[i, 'ПЖ sum'] = calc_table.at[i - 1, 'ПЖ sum'] * (1 + calc_table.at[i, 'netto'])
    calc_table.at[i, 'ПЖ prob'] = calc_table.at[i, 'ПЖ sum'] * calc_table.at[i, 'live'] * calc_table.at[i, 'noterm']
    calc_table.at[i, 'СР sum'] = calc_table.at[i - 1, 'СР sum'] * (1 + calc_table.at[i, 'netto'])
    calc_table.at[i, 'СР prob'] = calc_table.at[i, 'СР sum'] * calc_table.at[i, 'live'] * calc_table.at[i, 'noterm']
    calc_table.at[i, 'ДИ sum'] = calc_table.at[i, 'Остаток MOP'] / (Pens_Term - (calc_table.at[i, 'Год'] - DoR)) if Pension_type == 'ДИ' else 0.0
    calc_table.at[i, 'ДИ prob'] = calc_table.at[i, 'ДИ sum'] * calc_table.at[i, 'live'] * calc_table.at[i, 'noterm']
    calc_table.at[i, 'Остаток EOP'] = max(0, calc_table.at[i, 'Остаток MOP'] - calc_table.loc[i, ['ЕВ sum', 'ПЖ sum', 'СР sum', 'ДИ sum']].sum())
    calc_table.at[i, 'Админ sum'] = Admin * (1 + inflation)**(calc_table.at[i-1, 'Год']) if calc_table.loc[i, ['ЕВ sum', 'ЕВ prob', 'ПЖ sum', 'ПЖ prob', 'СР sum', 'СР prob']].sum() > 0.01 else 0.0
    calc_table.at[i, 'Админ prob'] = calc_table.at[i, 'Админ sum'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
  # 6) Расчёт summary и выгрузка в excel
  EV_total = (calc_table['Discont factor'] * calc_table['ЕВ prob']).sum()
  PZ_total = (calc_table['Discont factor'] * calc_table['ПЖ prob']).sum()
  SR_total = (calc_table['Discont factor'] * calc_table['СР prob']).sum()
  DI_total = (calc_table['Discont factor'] * calc_table['ДИ prob']).sum()

  Nasled_total = (calc_table['Discont factor'] * calc_table['Наследники prob']).sum()
  Vykup_total = (calc_table['Discont factor'] * calc_table['Выкупная prob']).sum()
  Vznos_total = (calc_table['Discont factor'] * calc_table['Взносы prob']).sum()
  VF_total = (calc_table['Discont factor'] * (calc_table['ВФ пост prob'] + calc_table['ВФ перем prob'])).sum()

  Admin_total = (calc_table['Discont factor'] * calc_table['Админ prob']).sum()
  UK_total = (calc_table['Discont factor'] * (calc_table['УК усп prob'] + calc_table['УК упр prob'])).sum()
  SD_total = (calc_table['Discont factor'] * calc_table['СД prob']).sum()
  ASV_total = (calc_table['Discont factor'] * calc_table['АСВ prob']).sum()

  Earned_total = Ostatok + Vznos_total - (EV_total + PZ_total + SR_total + DI_total) - (Nasled_total + Vykup_total) - (Admin_total + UK_total + SD_total + ASV_total)
  Earned_total_2 = VF_total - Admin_total - (UK_total + SD_total) if system == 'new' else VF_total - Admin_total

  Summary = pd.DataFrame({'Категория':['Пенсии']*4 + ['Выплаты']*4 + ['Расходы'] * 4 + ['Фин рез'] * 2,
          'Показатель':['ЕВ', 'ПЖ', 'СР', 'ДИ', 'Наслед', 'Выкупная', 'Взносы', 'ВФ', 'Админ', 'УК', 'СД', 'АСВ', 'Фонд заработал 1', 'Фонд заработал 2'],
          'Сумма':[EV_total, PZ_total, SR_total, DI_total, Nasled_total, Vykup_total, Vznos_total, VF_total, Admin_total, UK_total, SD_total, ASV_total, Earned_total, Earned_total_2]})
  # Запись в excel
  save_dir = file_path + "Итоги итераций/НПО_точка_" + str(mt_num) + ".xlsx"
  with pd.ExcelWriter(save_dir, engine = 'openpyxl') as writer:
      Summary.to_excel(writer, sheet_name = 'Свод итогов', index = False)
      calc_table.to_excel(writer, sheet_name = 'Потоки детально', index = False)
  crex.format_output(writer, save_dir)
  return Summary, calc_table

In [ ]:
# @title Функция расчёта
def NPO_model_month(gender, age, Pension_type, Ostatok, Vznos, mt_num, lapce = 0.005, death = 0.0, system = 'new'):
  Admin = 2500.0
  Vznos = 1000.0
  # % Доходностей:
  brutto = 0.0
  inflation = 0.042
  # % Вознаграждений от СЧА
  VF_pct, UK_pct, SD_pct, ASV_pct = 0.005, 0.001, 0.003, 0.000125
  # % Вознаграждений от ИД
  VF_inv_pct, UK_inv_pct = 0.15, 0.03
  # Прочее
  Tarif = 11 * 12
  Pens_Term = 10 * 12
  # Расчётные параметры:
  DoR = (60 - age if gender == 'М' else 55 - age) * 12 + 11
  DoTO = DoR + Pens_Term if Pension_type in ['ДИ', 'СР'] else DoR if Pension_type == 'ЕВ' else 100 * 12

  # 1) Исходные данные таблицы смертности и данные ставок КБД
  calc_table = death_props.query("`Age (Y)` >= @age and `Age (Y)` < 111").rename(columns = {'Age (Y)' :'Age'})
  calc_table = calc_table[['Age', 'q_x (M)']] if age == 'M' else calc_table[['Age', 'q_x (F)']]
  calc_table = calc_table.rename(columns = {calc_table.columns[-1] : 'Death'})
  calc_table['Год'] = range(calc_table.shape[0])
  calc_table['noterm'] = 1.0
  calc_table = calc_table.merge(KBD_data[['Y','МСФО 17']].rename(columns = {'Y':'Год', 'МСФО 17':'brutto'}), how = 'left', on = 'Год')
  calc_table['live'] = 1.0
  for i in range(1, len(calc_table)):
      calc_table.loc[i, 'live'] = (1-calc_table.loc[i, 'Death'])*calc_table.loc[i-1, 'live']
  calc_table['netto'] = calc_table['brutto']*(1-VF_inv_pct) - ASV_pct/12 - VF_pct/12 if system == 'new' else \
    (calc_table['brutto']*(1-UK_inv_pct) - VF_pct - UK_pct - SD_pct - ASV_pct)*(1-VF_inv_pct)
  calc_table = calc_table[['Год', 'Age', 'noterm', 'Death', 'live', 'brutto', 'netto']]
  calc_table['Discont factor'] = 1.0

  # Помесячный пересчет таблицы
  calc_table_month = year_month.merge(calc_table, on = 'Год', how = 'left')#.to_excel(file_path + "Тест.xlsx")
  calc_table_month['Death'] /= 12
  for i in range(1, len(calc_table_month)):
      calc_table_month.loc[i, 'live'] = (1-calc_table_month.loc[i, 'Death'])*calc_table_month.loc[i-1, 'live']
  calc_table_month['brutto'] = calc_table_month['brutto'].apply(lambda x: (1+x)**(1/12) - 1)
  calc_table_month['netto'] = calc_table_month['brutto']*(1-VF_inv_pct) - ASV_pct/12 - VF_pct/12 if system == 'new' else \
    (calc_table_month['brutto']*(1-UK_inv_pct) - VF_pct/12 - UK_pct/12 -SD_pct/12 - ASV_pct/12)*(1-VF_inv_pct)
  calc_table_month.loc[:12, 'Discont factor'] = 1.0
  for i in range(12, len(calc_table_month)):
    calc_table_month.loc[i, 'Discont factor'] = calc_table_month.loc[i - 1, 'Discont factor'] / (1 + calc_table_month.loc[i, 'brutto'])
  calc_table = calc_table_month.reset_index(drop = True)
  for i in range(12, len(calc_table)):
    calc_table.at[i, 'noterm'] = calc_table.at[i - 1, 'noterm']*(1-lapce/12)
  # 2) Расчёты нулевого года
  calc_table.at[11, 'Остаток EOP'] = Ostatok

  # 3) Расчёты с 1 года до выхода на пенсию
  for i in range(12, DoR + 1):
    calc_table.at[i, 'Остаток BOP'] = calc_table.at[i - 1, 'Остаток EOP']
    calc_table.at[i, 'ИД brutto sum'] = calc_table.at[i, 'Остаток BOP'] * calc_table.at[i, 'brutto']
    calc_table.at[i, 'ИД brutto prob'] = calc_table.at[i, 'ИД brutto sum'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'ИД netto prob'] = calc_table.at[i, 'Остаток BOP'] * calc_table.at[i, 'netto'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'ВФ пост prob'] = VF_pct/12 * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'УК усп prob'] = UK_inv_pct * calc_table.at[i, 'ИД brutto prob']
    calc_table.at[i, 'УК упр prob'] = UK_pct/12 * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'СД prob'] = SD_pct/12 * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'АСВ prob'] = ASV_pct/12 * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'ВФ перем prob'] = VF_inv_pct * (calc_table.at[i, 'ИД brutto prob'] if system == 'new' else calc_table.at[i, 'ИД brutto prob'] - \
      calc_table.loc[i, ['ВФ пост prob', 'УК усп prob', 'УК упр prob', 'СД prob', 'АСВ prob']].sum())

    calc_table.at[i, 'Взносы sum'] = Vznos/12
    calc_table.at[i, 'Взносы prob'] = calc_table.at[i, 'Взносы sum'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'Админ sum'] = Admin/12 * (1 + inflation)**(calc_table.at[i-12, 'Год'])
    calc_table.at[i, 'Админ prob'] = calc_table.at[i, 'Админ sum'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']

    calc_table.at[i, 'Остаток MOP'] = calc_table.at[i, 'Остаток BOP'] * (1 + calc_table.at[i, 'netto']) + calc_table.at[i, 'Взносы sum']
    calc_table.at[i, 'Наследники sum'] = calc_table.at[i, 'Остаток MOP']
    calc_table.at[i, 'Наследники prob'] = calc_table.at[i, 'Наследники sum'] * calc_table.at[i, 'Death'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']*(1-lapce/2)
    calc_table.at[i, 'Выкупная sum'] = calc_table.at[i, 'Остаток MOP']
    calc_table.at[i, 'Выкупная prob'] = calc_table.at[i, 'Выкупная sum'] * lapce * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm'] * (1 - calc_table.at[i, 'Death']/2)
  # Прочее
    for cl in ['ЕВ sum', 'ЕВ prob', 'ПЖ sum', 'ПЖ prob', 'СР sum', 'СР prob', 'ДИ sum', 'ДИ prob']:
      calc_table.at[i, cl] = 0.0
    calc_table.at[i, 'Остаток EOP'] = max(0, calc_table.at[i, 'Остаток MOP'])
  # 4) Расчёты в момент выхода на пенсию
  calc_table.at[DoR, 'ЕВ sum'] = calc_table.at[DoR, 'Остаток MOP'] if Pension_type == 'ЕВ' else 0.0
  calc_table.at[DoR, 'ЕВ prob'] = calc_table.at[DoR, 'ЕВ sum'] * calc_table.at[DoR, 'live'] * calc_table.at[DoR, 'noterm']
  calc_table.at[DoR, 'ПЖ sum'] = calc_table.at[DoR, 'Остаток MOP'] / Tarif if Pension_type == 'ПЖ' else 0.0
  calc_table.at[DoR, 'ПЖ prob'] = calc_table.at[DoR, 'ПЖ sum'] * calc_table.at[DoR, 'live'] * calc_table.at[DoR, 'noterm']
  calc_table.at[DoR, 'СР sum'] = calc_table.at[DoR, 'Остаток MOP'] / Pens_Term if Pension_type == 'СР' else 0.0
  calc_table.at[DoR, 'СР prob'] = calc_table.at[DoR, 'СР sum'] * calc_table.at[DoR, 'live'] * calc_table.at[DoR, 'noterm']
  calc_table.at[DoR, 'ДИ sum'] = calc_table.at[DoR, 'Остаток MOP'] / Pens_Term if Pension_type == 'ДИ' else 0.0
  calc_table.at[DoR, 'ДИ prob'] = calc_table.at[DoR, 'ДИ sum'] * calc_table.at[DoR, 'live'] * calc_table.at[DoR, 'noterm']
  calc_table.at[DoR, 'Остаток EOP'] = max(0, calc_table.at[DoR, 'Остаток MOP'] - calc_table.loc[DoR, ['ЕВ sum', 'ПЖ sum', 'СР sum', 'ДИ sum']].sum())


  # 5) Расчёты после выхода на пенсию
  for i in range(DoR + 1, DoTO):
    calc_table.at[i, 'noterm'] = calc_table.at[i - 1, 'noterm']
    calc_table.at[i, 'Остаток BOP'] = calc_table.at[i - 1, 'Остаток EOP']
    calc_table.at[i, 'ИД brutto sum'] = calc_table.at[i, 'Остаток BOP'] * calc_table.at[i, 'brutto']
    calc_table.at[i, 'ИД brutto prob'] = calc_table.at[i, 'ИД brutto sum'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'ИД netto prob'] = calc_table.at[i, 'Остаток BOP'] * calc_table.at[i, 'netto'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'ВФ пост prob'] = VF_pct/12 * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'УК усп prob'] = UK_inv_pct * calc_table.at[i, 'ИД brutto prob']
    calc_table.at[i, 'УК упр prob'] = UK_pct/12 * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'СД prob'] = SD_pct/12 * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'АСВ prob'] = ASV_pct/12 * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'ВФ перем prob'] = VF_inv_pct * (calc_table.at[i, 'ИД brutto prob'] if system == 'new' else calc_table.at[i, 'ИД brutto prob'] - \
      calc_table.loc[i, ['ВФ пост prob', 'УК усп prob', 'УК упр prob', 'СД prob', 'АСВ prob']].sum())

    calc_table.at[i, 'Взносы sum'] = 0.0
    calc_table.at[i, 'Взносы prob'] = 0.0

    calc_table.at[i, 'Остаток MOP'] = calc_table.at[i, 'Остаток BOP'] * (1 + calc_table.at[i, 'netto']) + calc_table.at[i, 'Взносы sum']
    calc_table.at[i, 'Наследники sum'] = calc_table.at[i, 'Остаток MOP'] if Pension_type != 'ПЖ' else 0
    calc_table.at[i, 'Наследники prob'] = calc_table.at[i, 'Наследники sum'] * calc_table.at[i, 'Death'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
    calc_table.at[i, 'Выкупная sum'] = 0.0
    calc_table.at[i, 'Выкупная prob'] = 0.0

    calc_table.at[i, 'ЕВ sum'] = 0.0
    calc_table.at[i, 'ЕВ prob'] = 0.0
    calc_table.at[i, 'ПЖ sum'] = calc_table.at[i - 1, 'ПЖ sum'] * (1 + calc_table.at[i, 'netto'])
    calc_table.at[i, 'ПЖ prob'] = calc_table.at[i, 'ПЖ sum'] * calc_table.at[i, 'live'] * calc_table.at[i, 'noterm']
    calc_table.at[i, 'СР sum'] = calc_table.at[i - 1, 'СР sum'] * (1 + calc_table.at[i, 'netto'])
    calc_table.at[i, 'СР prob'] = calc_table.at[i, 'СР sum'] * calc_table.at[i, 'live'] * calc_table.at[i, 'noterm']
    calc_table.at[i, 'ДИ sum'] = calc_table.at[i, 'Остаток MOP'] / (Pens_Term - (calc_table.at[i, 'Год'] - DoR)) if Pension_type == 'ДИ' else 0.0
    calc_table.at[i, 'ДИ prob'] = calc_table.at[i, 'ДИ sum'] * calc_table.at[i, 'live'] * calc_table.at[i, 'noterm']
    calc_table.at[i, 'Остаток EOP'] = max(0, calc_table.at[i, 'Остаток MOP'] - calc_table.loc[i, ['ЕВ sum', 'ПЖ sum', 'СР sum', 'ДИ sum']].sum())
    calc_table.at[i, 'Админ sum'] = Admin/12 * (1 + inflation)**(calc_table.at[i-12, 'Год']) if calc_table.loc[i, ['ЕВ sum', 'ЕВ prob', 'ПЖ sum', 'ПЖ prob', 'СР sum', 'СР prob']].sum() > 0.01 else 0.0
    calc_table.at[i, 'Админ prob'] = calc_table.at[i, 'Админ sum'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
  # 6) Расчёт summary и выгрузка в excel
  EV_total = (calc_table['Discont factor'] * calc_table['ЕВ prob']).sum()
  PZ_total = (calc_table['Discont factor'] * calc_table['ПЖ prob']).sum()
  SR_total = (calc_table['Discont factor'] * calc_table['СР prob']).sum()
  DI_total = (calc_table['Discont factor'] * calc_table['ДИ prob']).sum()

  Nasled_total = (calc_table['Discont factor'] * calc_table['Наследники prob']).sum()
  Vznos_total = (calc_table['Discont factor'] * calc_table['Взносы prob']).sum()
  VF_total = (calc_table['Discont factor'] * (calc_table['ВФ пост prob'] + calc_table['ВФ перем prob'])).sum()
  # Расчёты выкупной суммы:
  temp_tab = calc_table.groupby('Год', as_index = False).agg({'Discont factor':'last', 'Выкупная prob':'last'})
  Vykup_total = (temp_tab['Discont factor'] * temp_tab['Выкупная prob']).sum()

  Admin_total = (calc_table['Discont factor'] * calc_table['Админ prob']).sum()
  UK_total = (calc_table['Discont factor'] * (calc_table['УК усп prob'] + calc_table['УК упр prob'])).sum()
  SD_total = (calc_table['Discont factor'] * calc_table['СД prob']).sum()
  ASV_total = (calc_table['Discont factor'] * calc_table['АСВ prob']).sum()

  Earned_total = Ostatok + Vznos_total - (EV_total + PZ_total + SR_total + DI_total) - (Nasled_total + Vykup_total) - (Admin_total + UK_total + SD_total + ASV_total)
  Earned_total_2 = VF_total - Admin_total - (UK_total + SD_total) if system == 'new' else VF_total - Admin_total

  Summary = pd.DataFrame({'Категория':['Пенсии']*4 + ['Выплаты']*4 + ['Расходы'] * 4 + ['Фин рез'] * 2,
          'Показатель':['ЕВ', 'ПЖ', 'СР', 'ДИ', 'Наслед', 'Выкупная', 'Взносы', 'ВФ', 'Админ', 'УК', 'СД', 'АСВ', 'Фонд заработал 1', 'Фонд заработал 2'],
          'Сумма':[EV_total, PZ_total, SR_total, DI_total, Nasled_total, Vykup_total, Vznos_total, VF_total, Admin_total, UK_total, SD_total, ASV_total, Earned_total, Earned_total_2]})

  calc_table = calc_table.query("Год != 0")

  return Summary, calc_table

In [ ]:
def calc_all(gender, age, Pension_type, Ostatok, Vznos, mt_num, lapce = 0.005, death = 0.0, system = 'new'):
  Summary, calc_table = NPO_model(gender, age, Pension_type, Ostatok, Vznos, mt_num, system = 'new')
  Summary_mn, calc_table_mn = NPO_model_month(gender, age, Pension_type, Ostatok, Vznos, mt_num, system = 'new')
  Summary = Summary.merge(Summary_mn.rename(columns = {'Сумма':'Сумма помесячно'}), on = ['Категория', 'Показатель'], how = 'left')
  Summary['Разность'] = Summary['Сумма помесячно'] - Summary['Сумма']
  Summary['Разность в %'] = Summary['Разность'] / Summary['Сумма']
  # Запись в excel
  save_dir = file_path + "Итоги итераций/НПО_точка_" + str(mt_num) + ".xlsx"
  with pd.ExcelWriter(save_dir, engine = 'openpyxl') as writer:
      Summary.to_excel(writer, sheet_name = 'Свод итогов', index = False)
      calc_table.to_excel(writer, sheet_name = 'Потоки по годам', index = False)
      calc_table_mn.to_excel(writer, sheet_name = 'Потоки помесячно', index = False)

  crex.format_output(writer, save_dir)
  return Summary